In [ ]:
import multiprocessing
import psutil
import sys

import geopandas as gpd
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from railrailrail.network.stage import Stage
from railrailrail.railgraph import RailGraph

from analysis.analysis_helpers import (
    get_stage_journeys,
    make_stage_journeys_dataframe,
    contains_point,
    get_station_agg_stats,
    get_planning_area_to_region_df,
)

# Do not truncate DataFrames
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

minimum_available_ram_GB = 3
if psutil.virtual_memory().available < minimum_available_ram_GB * 1e9:
    print(
        "Warning: Less than %dGB RAM available. This program may crash with an Out-Of-Memory error."
        % minimum_available_ram_GB,
        file=sys.stderr,
    )


planning_areas = gpd.read_file("MasterPlan2019PlanningAreaBoundaryNoSea.geojson")
planning_areas["Planning Area"] = planning_areas["Description"].apply(
    lambda description: description.split("<td>", 1)[-1].split("</td>", 1)[0].strip()
)

planning_area_to_region_df = get_planning_area_to_region_df()


stages = list(Stage.stages)
journeys_by_stage: dict[str, dict[str, pd.DataFrame | RailGraph]] = dict()
station_agg_stats_by_stage: dict[str, pd.DataFrame] = dict()


with multiprocessing.get_context("fork").Pool(
    None
) as pool:  # https://bugs.python.org/issue33725 https://stackoverflow.com/a/72605750
    for stage, journeys, rail_graph in pool.imap_unordered(get_stage_journeys, stages):
        journeys_by_stage[stage] = {
            "df": make_stage_journeys_dataframe(stage, journeys),
            "rail_graph": rail_graph,
        }
        station_agg_stats_by_stage[stage] = get_station_agg_stats(
            df=journeys_by_stage[stage]["df"],
            rail_graph=journeys_by_stage[stage]["rail_graph"],
        )
        # Stations within interchanges have different geographic coordinates.
        # This causes them to have different average circuity. For example, see BP1/NS4/JS1 Choa Chu Kang.

        station_agg_stats_by_stage[stage]["planning_area"] = station_agg_stats_by_stage[
            stage
        ].apply(
            lambda row: next(
                (
                    feature["Planning Area"]
                    for _, feature in planning_areas.iterrows()
                    if contains_point(
                        feature["geometry"],
                        row["latitude"],
                        row["longitude"],
                    )
                ),
                None,
            ),
            axis=1,
        )

        station_agg_stats_by_stage[stage] = station_agg_stats_by_stage[stage].merge(
            planning_area_to_region_df, on="planning_area", how="left"
        )

In [ ]:
# Journeys between only stations that were around in 2003.
old_stations = list(journeys_by_stage["nel"]["df"]["start"].unique())
old_stations_df = pd.DataFrame({"station": old_stations})

recent_stages = set()
start = False
for stage in journeys_by_stage.keys():
    if stage == "nel":
        start = True
    if not start:
        continue
    recent_stages.add(stage)

journeys_by_stage_from_nel: dict[str, dict[str, pd.DataFrame | RailGraph]] = dict()
station_agg_stats_by_stage_from_nel: dict[str, pd.DataFrame] = dict()


def get_stage_journeys_cumulative(stage: str):
    if stage not in recent_stages:
        return None
    return (
        journeys_by_stage[stage]["df"]
        .merge(old_stations_df, left_on="start", right_on="station")
        .merge(old_stations_df, left_on="end", right_on="station")
    )


for stage in stages:
    df = get_stage_journeys_cumulative(stage)
    if df is None:
        continue
    journeys_by_stage_from_nel[stage] = dict()
    journeys_by_stage_from_nel[stage]["rail_graph"] = journeys_by_stage[stage][
        "rail_graph"
    ]
    journeys_by_stage_from_nel[stage]["df"] = df

    station_agg_stats_by_stage_from_nel[stage] = get_station_agg_stats(
        df=journeys_by_stage_from_nel[stage]["df"],
        rail_graph=journeys_by_stage_from_nel[stage]["rail_graph"],
    )
    # Stations within interchanges have different geographic coordinates.
    # This causes them to have different average circuity. For example, see BP1/NS4 Choa Chu Kang.

    station_agg_stats_by_stage_from_nel[stage]["planning_area"] = (
        station_agg_stats_by_stage_from_nel[stage].apply(
            lambda row: next(
                (
                    feature["Planning Area"]
                    for _, feature in planning_areas.iterrows()
                    if contains_point(
                        feature["geometry"],
                        row["latitude"],
                        row["longitude"],
                    )
                ),
                None,
            ),
            axis=1,
        )
    )

    station_agg_stats_by_stage_from_nel[stage] = station_agg_stats_by_stage_from_nel[
        stage
    ].merge(planning_area_to_region_df, on="planning_area", how="left")

In [ ]:
def plot_planning_area_agg_stats_scatter(
    station_agg_stats_by_stage: dict[str, pd.DataFrame], stage: str, show=False
):
    stage_description, stage_timestamp = Stage.stages_info[stage]
    planning_area_agg_stats = (
        station_agg_stats_by_stage[stage]
        .copy()
        .groupby(by=["planning_area", "region"])
        .mean(numeric_only=True)
        .reset_index()
        .sort_values(by=["path_distance_mean"])
    )
    planning_area_agg_stats["size"] = 10  # Chart dot size
    # print(
    #     "Number of active planning areas:",
    #     planning_area_agg_stats["planning_area"].nunique(),
    # )

    fig = px.scatter(
        planning_area_agg_stats,
        title=(
            "Mean Circuity against Mean Fastest Path Length (Averaged By Planning Area)"
            "<br />"
            f"{stage_description} @ {stage_timestamp.strftime("%d %B %Y")}"
        ),
        x="path_distance_mean",
        y="circuity_mean",
        range_x=(
            int(planning_area_agg_stats["path_distance_mean"].min() * 0.95),
            int(planning_area_agg_stats["path_distance_mean"].max() * 1.05),
        ),
        range_y=(
            planning_area_agg_stats["circuity_mean"].min() - 0.1,
            planning_area_agg_stats["circuity_mean"].max() + 0.1,
        ),
        labels={
            "path_distance_mean": "Mean Fastest Path Length",
            "circuity_mean": "Mean Circuity",
            "planning_area": "Planning Area",
            "region": "Region",
        },
        size="size",
        color="region",
        color_discrete_map={
            "CENTRAL": "#ffa15a",
            "WEST": "#636efa",
            "EAST": "#00cc96",
            "NORTH-EAST": "fuchsia",
            "NORTH": "#ef553b",
        },
        text="planning_area",
        hover_name="planning_area",
        hover_data={"size": False},
        # trendline="lowess",  # Locally WEighted Scatterplot Smoothing (LOWESS)
        # trendline_scope="overall",
        log_x=False,
        log_y=False,
        size_max=10,
        template="plotly_dark",
    )

    fig.add_hline(
        y=planning_area_agg_stats["circuity_mean"].mean(),
        line_width=2,
        line_dash="dash",
        line_color="red",
    )
    fig.add_vline(
        x=planning_area_agg_stats["path_distance_mean"].mean(),
        line_width=2,
        line_dash="dash",
        line_color="red",
    )

    fig.add_annotation(
        x=planning_area_agg_stats["path_distance_mean"].max(),
        y=planning_area_agg_stats["circuity_mean"].mean(),
        text=f"Mean: {planning_area_agg_stats["circuity_mean"].mean():.2f}",
        showarrow=False,
        yshift=10,
    )  # Median Mean Circuity across all Planning Areas

    fig.add_annotation(
        x=planning_area_agg_stats["path_distance_mean"].mean(),
        y=planning_area_agg_stats["circuity_mean"].max(),
        text=f"Mean: {planning_area_agg_stats["path_distance_mean"].mean() / 1000 :.1f}km",
        showarrow=False,
        xshift=50,
    )  # Median Mean Fastest Path Length across all Planning Areas

    fig.update_layout(
        title={"x": 0.5},
        dragmode="pan",
        xaxis=dict(
            tickformat="",
            ticksuffix="m",
            tickfont=dict(color="white", family="Arial Black"),
        ),
        yaxis=dict(
            tickfont=dict(color="white", family="Arial Black"),
        ),
    )
    fig.update_traces(
        textposition="top center",
        textfont=dict(size=13, color="white", family="Arial"),
    )

    if show:
        fig.show()
    return planning_area_agg_stats, fig


for stage in ["teck_lee", "cg_tel_c"]:
    planning_area_agg_stats, fig = plot_planning_area_agg_stats_scatter(
        station_agg_stats_by_stage, stage, show=True
    )
    print(fig.to_html(full_html=False, include_plotlyjs="cdn"))
    print()

for stage in ["teck_lee", "cg_tel_c"]:
    planning_area_agg_stats, fig = plot_planning_area_agg_stats_scatter(
        station_agg_stats_by_stage_from_nel, stage, show=True
    )
    print(fig.to_html(full_html=False, include_plotlyjs="cdn"))
    print()

In [ ]:
df = journeys_by_stage["phase_1_2"]["df"]
print(
    "%.1f%% of journeys as of Phase 1b are between adjacent stations."
    % (len(df.query("total_cost != 0 and circuity == 1.0")) / len(df) * 100,)
)

df = journeys_by_stage["teck_lee"]["df"]
print(
    "%.1f%% of journeys as of Teck Lee Station launch are between adjacent stations."
    % (len(df.query("total_cost != 0 and circuity == 1.0")) / len(df) * 100,)
)

In [ ]:
def plot_agg_stats_map_multi(
    station_agg_stats_by_stage: dict[str, pd.DataFrame],
    stages: list[str],
    color: str,
    hover_data: list[str],
    show: bool = False,
):
    station_agg_stats = pd.DataFrame()
    for stage in stages:
        df = station_agg_stats_by_stage[stage].copy()
        df["full_name"] = df.apply(
            lambda row: row["start"] + " " + row["station_name"], axis=1
        )
        df["stage_description"] = Stage.stages_info[stage][0]
        df["stage_timestamp"] = Stage.stages_info[stage][1].strftime("%d %B %Y")
        station_agg_stats = pd.concat([station_agg_stats, df], axis=0)

    max_circuity_mean = station_agg_stats["circuity_mean"].max()
    max_circuity_median = station_agg_stats["circuity_median"].max()
    max_circuity = max(max_circuity_mean, max_circuity_median)

    max_path_distance_mean = station_agg_stats["path_distance_mean"].max()
    max_path_distance_median = station_agg_stats["path_distance_median"].max()
    max_path_distance = max(max_path_distance_mean, max_path_distance_median)

    station_agg_stats["marker_size"] = 1
    fig = px.scatter_mapbox(
        station_agg_stats,
        title=f"{"Mean Circuity" if "circuity" in color else "Mean Fastest Path Length"}",
        lat="latitude",
        lon="longitude",
        animation_frame="stage_timestamp",
        hover_name="full_name",
        hover_data=hover_data,
        color=color,
        color_continuous_scale=px.colors.sequential.Jet,
        range_color=(
            (1, max_circuity) if "circuity" in color else (8000, max_path_distance)
        ),
        mapbox_style="carto-darkmatter",
        template="plotly_dark",
        zoom=9,
        labels={
            "path_distance_mean": "Mean Fastest Path Length",
            "circuity_mean": "Mean Circuity",
            "planning_area": "Planning Area",
            "region": "Region",
        },
        size="marker_size",
        size_max=9,
        opacity=1,
        center={"lat": 1.330270, "lon": 103.851959},
    )
    fig["layout"].pop("updatemenus")
    fig.update_layout(
        title={"x": 0.5, "y": 0.95},
        margin=dict(t=50, b=0),
        sliders=[
            {
                "currentvalue": {
                    "font": {"size": 20},
                    "prefix": None,
                    "visible": True,
                    "xanchor": "center",
                },
                "transition": {"duration": 0, "easing": "linear"},
                "pad": {"t": 0, "l": 0},
            }
        ],
    )
    if "circuity" in color:
        fig.update_coloraxes(colorbar_title="Mean Circuity")
    else:
        fig.update_coloraxes(
            colorbar_ticksuffix="m", colorbar_title="Mean Fastest Path Length"
        )
    fig.update_coloraxes(showscale=True)

    if show:
        fig.show()
    return station_agg_stats, fig


for agg_stats_by_stage in [
    station_agg_stats_by_stage,
    station_agg_stats_by_stage_from_nel,
]:
    for color in ["path_distance_mean", "circuity_mean"]:
        station_agg_stats, fig = plot_agg_stats_map_multi(
            agg_stats_by_stage,
            [
                "nel",
                "pglrt_east_loop_and_sklrt_west_loop",
                "dtl_2",
                "teck_lee",
                "cg_tel_c",
            ],
            color=color,
            hover_data={
                "circuity_mean": True,
                "path_distance_mean": True,
                "region": True,
                "planning_area": True,
                "marker_size": False,
                "stage_timestamp": False,
            },
            show=True,
        )
        print(fig.to_html(full_html=False, include_plotlyjs="cdn", auto_play=False))

In [ ]:
def plot_agg_stats_map(
    stage: str, color: str, hover_data: list[str], show: bool = False
):
    station_agg_stats = station_agg_stats_by_stage[stage].copy()
    station_agg_stats["full_name"] = station_agg_stats.apply(
        lambda row: row["start"] + " " + row["station_name"], axis=1
    )
    stage_description = Stage.stages_info[stage][0]
    stage_timestamp = Stage.stages_info[stage][1]

    max_circuity_mean = station_agg_stats["circuity_mean"].max()
    max_circuity_median = station_agg_stats["circuity_median"].max()
    max_circuity = max(max_circuity_mean, max_circuity_median)

    max_path_distance_mean = station_agg_stats["path_distance_mean"].max()
    max_path_distance_median = station_agg_stats["path_distance_median"].max()
    max_path_distance = max(max_path_distance_mean, max_path_distance_median)

    station_agg_stats["marker_size"] = 1
    fig = px.scatter_mapbox(
        station_agg_stats,
        title=(
            f"{"Mean Circuity" if "circuity" in color else "Mean Fastest Path Length"}"
            "<br />"
            f"{stage_description} @ {stage_timestamp.strftime("%d %B %Y")}"
        ),
        lat="latitude",
        lon="longitude",
        hover_name="full_name",
        hover_data=hover_data,
        color=color,
        color_continuous_scale=px.colors.sequential.Jet,
        range_color=(
            (1, max_circuity) if "circuity" in color else (8000, max_path_distance)
        ),
        mapbox_style="carto-darkmatter",
        template="plotly_dark",
        zoom=9,
        labels={
            "path_distance_mean": "Mean Fastest Path Length",
            "circuity_mean": "Mean Circuity",
            "planning_area": "Planning Area",
            "region": "Region",
        },
        size="marker_size",
        size_max=9,
        opacity=1,
        center={"lat": 1.330270, "lon": 103.851959},
    )
    fig["layout"].pop("updatemenus")
    fig.update_layout(
        title={"x": 0.5},
    )
    if "circuity" in color:
        fig.update_coloraxes(colorbar_title="Mean Circuity")
    else:
        fig.update_coloraxes(
            colorbar_ticksuffix="m", colorbar_title="Mean Fastest Path Length"
        )
    fig.update_coloraxes(showscale=True)

    if show:
        fig.show()
    return station_agg_stats, fig


station_agg_stats, fig = plot_agg_stats_map(
    "teck_lee",
    color="circuity_mean",
    hover_data={
        "circuity_mean": True,
        "path_distance_mean": True,
        "region": True,
        "planning_area": True,
        "marker_size": False,
    },
    show=True,
)
print(fig.to_html(full_html=False, include_plotlyjs="cdn", auto_play=False))

In [ ]:
def plot_agg_stats_grid(color: str, hover_data: list[str], show: bool = False):
    station_agg_stats = pd.DataFrame()
    for stage in stages:
        df = station_agg_stats_by_stage[stage].copy()
        df["full_name"] = df.apply(
            lambda row: row["start"] + " " + row["station_name"], axis=1
        )
        df["stage_description"] = Stage.stages_info[stage][0]
        df["stage_timestamp"] = Stage.stages_info[stage][1].strftime("%b %Y")
        station_agg_stats = pd.concat([station_agg_stats, df], axis=0)

    max_circuity_mean = station_agg_stats["circuity_mean"].max()
    max_circuity_median = station_agg_stats["circuity_median"].max()
    max_circuity = max(max_circuity_mean, max_circuity_median)

    max_path_distance_mean = station_agg_stats["path_distance_mean"].max()
    max_path_distance_median = station_agg_stats["path_distance_median"].max()
    max_path_distance = max(max_path_distance_mean, max_path_distance_median)

    station_agg_stats["marker_size"] = 1
    fig = px.scatter(
        station_agg_stats,
        title=f"{"Mean Circuity" if "circuity" in color else "Mean Fastest Path Length"}",
        y="latitude",
        x="longitude",
        range_y=[1.25, 1.46],
        range_x=[103.625, 104.02],
        animation_frame="stage_timestamp",
        hover_name="full_name",
        hover_data=hover_data,
        color=color,
        color_continuous_scale=px.colors.sequential.Jet,
        range_color=(
            (1, max_circuity) if "circuity" in color else (8000, max_path_distance)
        ),
        template="plotly_dark",
        size="marker_size",
        size_max=8,
        opacity=1,
    )
    fig["layout"].pop("updatemenus")
    fig.update_layout(
        title={"x": 0.5},
        xaxis=dict(scaleanchor="y", scaleratio=1),
        dragmode="pan",
        sliders=[
            {
                "currentvalue": {
                    "font": {"size": 20},
                    "prefix": None,
                    "visible": True,
                    "xanchor": "center",
                },
                "transition": {"duration": 0, "easing": "linear"},
                "pad": {"t": 50, "l": 5},
            }
        ],
    )
    if "circuity" in color:
        fig.update_coloraxes(colorbar_title="Mean Circuity")
    else:
        fig.update_coloraxes(
            colorbar_ticksuffix="m", colorbar_title="Mean Fastest Path Length"
        )
    fig.update_coloraxes(showscale=True)

    # Change initial frame
    frame_num = next(
        i for i, frame in enumerate(fig.frames) if frame["name"] == "Aug 2024"
    )
    fig.layout["sliders"][0]["active"] = frame_num
    fig = go.Figure(
        data=fig["frames"][frame_num]["data"], frames=fig["frames"], layout=fig.layout
    )

    if show:
        fig.show(config={"scrollZoom": True})
    return station_agg_stats, fig


for color in ["path_distance_mean", "circuity_mean"]:
    station_agg_stats, fig = plot_agg_stats_grid(
        color=color,
        hover_data=["circuity_mean", "path_distance_mean", "region", "planning_area"],
        show=True,
    )
    print(fig.to_html(full_html=False, include_plotlyjs="cdn", auto_play=False))

In [ ]:
for stage in journeys_by_stage:
    stage_df = journeys_by_stage[stage]["df"].query("total_cost != 0")
    stage_mean_total_cost = int(stage_df["total_cost"].mean())
    stage_max_total_cost = int(stage_df["total_cost"].max())
    stage_total_cost_std_dev = int(stage_df["total_cost"].std())
    print(stage, stage_mean_total_cost, stage_max_total_cost, stage_total_cost_std_dev)

In [ ]:
median_price_per_sq_ft_per_lease_year = pd.DataFrame.from_dict(
    {
        "ANG MO KIO": 9.79,
        "BEDOK": 9.32,
        "BISHAN": 11.27,
        "BUKIT BATOK": 7.33,
        "BUKIT MERAH": 10.8,
        "BUKIT PANJANG": 7.08,
        "BUKIT TIMAH": 12.86,
        "CENTRAL AREA": 13.2,  # Not a planning area.
        "CHOA CHU KANG": 6.44,
        "CLEMENTI": 9.95,
        "GEYLANG": 10.53,
        "HOUGANG": 8,
        "JURONG EAST": 8.08,
        "JURONG WEST": 6.8,
        "KALLANG": 10.69,
        "MARINE PARADE": 12.5,
        "PASIR RIS": 7.63,
        "PUNGGOL": 7.11,
        "QUEENSTOWN": 11.22,
        "SEMBAWANG": 6.54,
        "SENGKANG": 6.97,
        "SERANGOON": 9.74,
        "TAMPINES": 8.34,
        "TOA PAYOH": 10.77,
        "WOODLANDS": 6.76,
        "YISHUN": 7.69,
    },
    orient="index",
).reset_index()
median_price_per_sq_ft_per_lease_year.columns = [
    "planning_area",
    "median_price_per_sq_ft_per_lease_year",
]

planning_area_agg_stats = (
    station_agg_stats_by_stage["teck_lee"]
    .copy()
    .groupby(by=["planning_area", "region"])
    .mean(numeric_only=True)
    .reset_index()
    .sort_values(by=["path_distance_mean"])
    .merge(median_price_per_sq_ft_per_lease_year, on="planning_area", how="inner")
)

hover_data = {
    "haversine_distance_median": True,
    "path_distance_median": True,
    "region": True,
    "planning_area": True,
}

fig = px.scatter_3d(
    planning_area_agg_stats,
    x="haversine_distance_median",
    y="path_distance_median",
    z="median_price_per_sq_ft_per_lease_year",
    color="region",
    range_x=[
        planning_area_agg_stats["haversine_distance_median"].max(),
        planning_area_agg_stats["haversine_distance_median"].min(),
    ],
    text="planning_area",
    hover_name="planning_area",
    hover_data=hover_data,
    height=850,
)
fig.update_layout(
    scene=dict(
        xaxis_title="Median Haversine (km)",
        yaxis_title="Median Path (km)",
        zaxis_title="$/sqft/yr",
    )
)
fig.show()